In [82]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

In [84]:
def GetNews_commercialtimes(response):
    soup = BeautifulSoup(response.text)
    url = soup.find('link')['href']
    print(url)
    ndf = pd.DataFrame(data = [{'TITLE':soup.find('h1', attrs={'class':'single-post-title'}).text,
                                'TIME':datetime.strptime(soup.find('meta', attrs={'property':'article:published_time'})['content'],'%Y-%m-%dT%H:%M:%S+00:00'),
                                # 'CATEGORY':soup.find('meta',attrs={'property':'article:section'})['content'],
                                'DESCRIPTION':soup.find('meta',attrs={'name':'description'})['content'],
                                'CONTENT':'\n'.join(i.text for i in soup.find('div',attrs={'class':'entry-content clearfix single-post-content'}).find_all('p')),
                                # 'KEYWORDS':soup.find('meta',{'name':'news_keywords'})['content'],
                                'FROM':soup.find('meta',{'property':'og:site_name'})['content'],
                                'LINK':soup.find('meta', {'property':'og:url'})['content']}],
                       columns = ['TITLE', 'TIME', 'DESCRIPTION', 'CONTENT', 'FROM', 'LINK']) 
    return ndf

In [10]:
# url = 'https://ctee.com.tw/news/stocks/654327.html'
# resp = requests.get(url)
# GetNews_chinatimes(resp)

https://cteecors.azureedge.net


,TITLE,TIME,DESCRIPTION,CONTENT,FROM,LINK
0,\n車用電子、LED不妙？專家示警這2檔,2022-06-04 07:27:14,「新小奇」不妙！專家示警兩檔成照妖鏡 台股近期漲多回檔，相較大型權值股修正，改由電子小兵接棒...,台股近期漲多回檔，相較大型權值股修正，改由電子小兵接棒演出，其中以新興產業、小股本、具獨特利...,工商時報,https://ctee.com.tw/news/stocks/654327.html


In [85]:
def GetLinks_commercialtimes(response):
    links = []
    soup = BeautifulSoup(response.text)
    for i in soup.find_all('h2'):
        url = i.find('a')['href']
        links.append(url)
    return links

In [68]:
# url = 'https://ctee.com.tw/tag/台積電'
# resp = requests.get(url)
# GetLinks_chinatimes(resp)

['https://ctee.com.tw/realtimenews/cna/654563.html',
 'https://ctee.com.tw/news/tech/654392.html',
 'https://ctee.com.tw/news/tech/654211.html',
 'https://ctee.com.tw/news/tech/654144.html',
 'https://ctee.com.tw/news/tech/653953.html',
 'https://ctee.com.tw/news/stocks/653644.html',
 'https://ctee.com.tw/news/global/653561.html',
 'https://ctee.com.tw/bookstore/magazine/653476.html',
 'https://ctee.com.tw/news/stocks/653438.html',
 'https://ctee.com.tw/news/stocks/653423.html']

In [86]:
def Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [89]:
def CrawlNews_commercialtimes(keywords, pages):
    links = []
    for i in range(pages):
        url = 'https://ctee.com.tw/tag/{}/page/{}'.format(keywords, i)
        print(url)
        resp = requests.get(url)
        links += GetLinks_commercialtimes(resp)
        links = list(set(links))  
        print(links)
        print('There are {} links in page {}.'.format(len(links),str(i)))

    responses = [Crawl(link) for link in links]

    list_of_dataframes = []
    for response in responses:
        try:
            ndf = GetNews_commercialtimes(response)
            list_of_dataframes.append(ndf)
        except:
            pass
    df = pd.concat(list_of_dataframes, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

In [90]:
df = CrawlNews_commercialtimes(keywords='台積電', pages=2)

https://ctee.com.tw/tag/台積電/page/0
['https://ctee.com.tw/news/tech/654144.html', 'https://ctee.com.tw/news/global/653561.html', 'https://ctee.com.tw/news/tech/654392.html', 'https://ctee.com.tw/news/tech/654211.html', 'https://ctee.com.tw/news/stocks/653644.html', 'https://ctee.com.tw/realtimenews/cna/654563.html', 'https://ctee.com.tw/news/tech/653953.html', 'https://ctee.com.tw/news/stocks/653423.html', 'https://ctee.com.tw/bookstore/magazine/653476.html', 'https://ctee.com.tw/news/stocks/653438.html']
There are 10 links in page 0.
https://ctee.com.tw/tag/台積電/page/1
['https://ctee.com.tw/news/tech/654144.html', 'https://ctee.com.tw/news/global/653561.html', 'https://ctee.com.tw/news/tech/654392.html', 'https://ctee.com.tw/news/tech/654211.html', 'https://ctee.com.tw/news/stocks/653644.html', 'https://ctee.com.tw/realtimenews/cna/654563.html', 'https://ctee.com.tw/news/tech/653953.html', 'https://ctee.com.tw/news/stocks/653423.html', 'https://ctee.com.tw/bookstore/magazine/653476.html

In [91]:
df

,TITLE,TIME,DESCRIPTION,CONTENT,FROM,LINK
0,\n台積電在美建廠縮時短片 網友：Amazing,2022-06-03 11:40:13,晶圓代工龍頭台積電斥資120億美元於美國亞利桑那州鳳凰城興建晶圓廠，台積電周四（2日）在職場...,晶圓代工龍頭台積電斥資120億美元於美國亞利桑那州鳳凰城興建晶圓廠，台積電周四（2日）在職場...,工商時報,https://ctee.com.tw/news/tech/654144.html
1,\n谷歌大單台積搶輸三星？外媒揭內幕,2022-06-02 06:05:44,台積電搶輸死敵？美巨頭新晶片下單韓廠 背後1大誘因擋不了 三星因4奈米良率不佳，迫使大客戶高...,三星因4奈米良率不佳，迫使大客戶高通將驍龍8 Gen1 Plus旗艦晶片轉由台積電4奈米生產...,工商時報,https://ctee.com.tw/news/global/653561.html
2,\n台積股東會聚焦漲價 劉德音對3奈米有信心,2022-06-04 21:00:00,台積電周三（8日）召開年度股東常會，由董事長劉德音主持。股東會主要議題為去年營業報告、盈餘分...,台積電周三（8日）召開年度股東常會，由董事長劉德音主持。股東會主要議題為去年營業報告、盈餘分...,工商時報,https://ctee.com.tw/news/tech/654392.html
3,\n全球晶圓代工廠營收 去年增31％,2022-06-03 17:00:00,市場研究機構Gartner（顧能）最新統計顯示，2021年全球晶圓代工產業營收規模首度突破千...,市場研究機構Gartner（顧能）最新統計顯示，2021年全球晶圓代工產業營收規模首度突破千...,工商時報,https://ctee.com.tw/news/tech/654211.html
4,\n防端午變盤？大咖賣544張台積結果曝,2022-06-02 07:51:39,大咖狠賣500張台積電賠千萬 網見細節驚呼：很強了 台積電近期走勢震盪，前陣子一度殺到波段低...,台積電近期走勢震盪，前陣子一度殺到波段低點505元，近期則一路反彈最高來到560元，部分買在...,工商時報,https://ctee.com.tw/news/stocks/653644.html
5,\n重量級股東會將登場 法人估台股區間震盪,2022-06-05 11:09:18,台股上周上揚286.35點，漲幅約1.76%，周線連3紅。法人表示，本周將有台積電等重量級企...,台股上周上揚286.35點，漲幅約1.76%，周線連3紅。法人表示，本周將有台積電等重量級企...,工商時報,https://ctee.com.tw/realtimenews/cna/654563.html
6,\n全球半導體設備Q1出貨 同期新高,2022-06-02 17:00:00,達246.9億美元，較去年同期增加5％，前三大設備採購地區分別為大陸、韓國、台灣。,年增5% ，全年可望再創紀錄 陸、韓、台廠商積極擴產，居採購前三強\n國際半導體產業協會（S...,工商時報,https://ctee.com.tw/news/tech/653953.html
7,\n連假將至買盤觀望 台積開低 早盤跌逾百點,2022-06-02 01:20:45,"台股加權指數2日開低在16,605.46點，權王台積電（2330）開低在544元，大型權值股...","台股加權指數2日開低在16,605.46點，權王台積電（2330）開低在544元，大型權值股...",工商時報,https://ctee.com.tw/news/stocks/653423.html
8,\n台股想空翻多 還有兩大魔王關待解,2022-06-02 02:08:19,台股近節情怯，連2天走低，專家提醒，台股多空節奏將緊跟那斯達克，因台灣電子股跌幅較美國科技股...,台股近節情怯，連2天走低，專家提醒，台股多空節奏將緊跟那斯達克，因台灣電子股跌幅較美國科技股...,工商時報,https://ctee.com.tw/news/stocks/653438.html
